### All imports

In [224]:
import random
from cryptography import fernet
from cryptography.fernet import Fernet
import sqlite3
import hashlib
from tkinter import *
from tkinter.ttk import *
from PIL import *
from tkinter import messagebox
from tkinter import colorchooser
import tkinter as tk

### Backend 

In [4]:
class generator:
    def __init__(self) -> None:
        self.characters = "azertyuiopqsdfghjklmwxcvbnAZERTYUIOPQSDFGHJKLMWXCVBN0123456789_$:)/!><#&-+@.?"
        self.length = 30 
        self.mdp = ""
        self.compteur = 0 
    
    def passgen(self):
        while self.compteur < self.length:
            word = self.characters[random.randint(0, len(self.characters)-1)] 
            self.mdp += word 
            self.compteur += 1
        return self.mdp

In [5]:
#This class is used to encrypt/decrypt the user password
class Encryption:

    def gen_key(self):
        #check if key already exist
        #if not (self.check_key()):
        key = Fernet.generate_key()
        print(key)
        with open ("key_store","wb") as k:
            k.write(key)

# This method check if the exist 
    def check_key(self):
        try:
            if(open("key_store", "rb").read()):
                return True
        except:
            return False 
#This method return the key
    def load_key (self):
        return (open("key_store","rb").read())

#This method encrypt the message
    def encrypt(self,message):
        key = self.load_key()
        encoded_message= message.encode()
        f = Fernet(key)
        enc_message = f.encrypt(encoded_message)
        return enc_message

#This method decrypt the message   
    def decrypt(self,message):
        key = self.load_key()
        f=Fernet(key)
        dec_message = f.decrypt(message)
        return dec_message

In [162]:
class appDb:
    def __init__(self):
        try:
            self.connect = sqlite3.connect("pwmdatabase.db")
            self.cursor = self.connect.cursor()
        except Exception as e:
            print(e)
    
    def createDataTable (self):
        sCreate = """
			CREATE TABLE IF NOT EXISTS appData (siteName varchar(300),siteUrl varchar(300),sitePassword text)
		"""
        self.cursor.execute(sCreate)
        self.connect.commit()

    def insertDataTable(self,sn,su,sp):
        enc = Encryption()
        enc_ps = enc.encrypt(sp)
        sInsert = """
            INSERT INTO appData (siteName,siteUrl,sitePassword) 
            VALUES (?,?,?)
        """
        self.cursor.execute(sInsert, (sn, su, enc_ps))
        self.connect.commit()
    
    def selectData(self,sname):
        enc = Encryption()
        select="SELECT * FROM appData WHERE siteName LIKE '%{}%'".format(sname)
        self.cursor.execute(select)
        c = self.cursor.fetchall()
        self.connect.commit()
        if not c:
            return ("")
        return c
        
    def searchPasswd (self, sname):
        enc = Encryption()
        resultsiteName = "SELECT * FROM appData WHERE siteName LIKE '%{}%'".format(sname)
        self.cursor.execute(resultsiteName)
        c = self.cursor.fetchall()
        self.connect.commit()
        if not c:
            return ("")

        dec_ps = enc.decrypt(c[0][2]).decode('utf-8')
        return (dec_ps)

    def deleteDataTable(self,sn):
        sDelete =("DELETE FROM appData WHERE siteName LIKE '%{}%'").format(sn)
        try:
            self.cursor.execute(sDelete)
            self.connect.commit()
        except Exception as e:
            print(e)

    def dropTable(self):
        sDrop = ('DELETE FROM appData') 
        try:
            self.cursor.execute(sDrop)
            self.connect.commit()
        except Exception as e:
            print(e)

    def printData (self):
        sPrint = """
            SELECT siteName, siteUrl FROM appData
        """
        self.cursor.execute(sPrint)
        self.connect.commit()
        alldata = self.cursor.fetchall()
        return alldata
       

In [333]:
class MasterDB:

    def __init__(self):
        try:
            self.connect = sqlite3.connect("pwmdatabase.db")
            self.cursor = self.connect.cursor()
        except Exception as e:
            print(e)

# This will create masterTable

    def createTable(self):
    		qCreate = """
    			CREATE TABLE IF NOT EXISTS masterTable (masterPass varchar(300))
    		"""
    		self.cursor.execute(qCreate)
    		self.connect.commit()

    # This will hash the password and insert it along with email entered

    def insertIntoTable(self, mp):
    	bytesMP = bytes(mp, 'utf-8')
    	hashedMP = hashlib.sha256(bytesMP).hexdigest()
    	qInsert = """
    		INSERT INTO masterTable (masterPass)
    		VALUES (?)
    	"""
    	self.cursor.execute(qInsert, [hashedMP])
    	self.connect.commit()

    # This will update the existing password(hashed) instead of making a new row in database

    def updateIntoTable(self,mp):
        bytesMP = bytes (mp ,'utf-8')
        hashedMP = hashlib.sha256(bytesMP).hexdigest()
        qUpdate = """
            UPDATE masterTable SET masterpass = (?)
            """
        self.cursor.execute(qUpdate, [hashedMP])
        self.connect.commit()

    # This is used in loginFrame.py to check the password with database

    def loginCheck(self,mp):
        bytesMP = bytes (mp ,'utf-8')
        hashedMP = hashlib.sha256(bytesMP).hexdigest()

        qSelect = """
    			SELECT * FROM masterTable
    		"""
        self.cursor.execute(qSelect)
        data = self.cursor.fetchall()
        print(data)
        if len(data) == 0:
            messagebox.showwarning("Login","You Are not register\n Please Sign up")
            return False

        if hashedMP == data[0][0]:
            return True
        return False


In [375]:
class StartPage:
    def __init__(self,master) -> None:
        self.master = master
        self.master.title("Login Page")
        self.master.geometry("720x480")
        self.master.maxsize(720,480)
        self.master.minsize (480,360)
        self.master.config(background='#000000')
        self.frame = tk.Frame(self.master,bg='#000000')
        self.btn = tk.Frame(self.frame,bg='#000000')
        self.frame.pack(expand=YES)
        self.db = MasterDB()
        
        #add a text
        label_title = tk.Label(self.frame,text="Welcome to my Password Manager", font=("Geneva",30), bg='#000000', fg='white')
        label_title.pack()
        sub_label_title = tk.Label(self.frame,text="Happy to count you among our users", font=("Geneva",12), bg='#000000', fg='white')
        sub_label_title.pack()

        #Add a button 
        log_button= Button(self.btn, text ='Login',command= self.login).pack(side=LEFT, padx=15, pady=30) #add command=fonction 
        sign_button=Button(self.btn, text ='Sing up',command=self.sign_up).pack(side=RIGHT, padx=15, pady=30)
        self.btn.pack(expand=YES)

    def login (self):
        self.passwd = Toplevel(self.master)
        self.passwd.title("Login")
        self.passwd.config(background='#000000')
        self.passwd.geometry("400x200")

        #Create Label Frame
        self.lbl_title = tk.Label(self.passwd,text="Password", font=("Geneva",10), bg='#000000', fg='white')
        self.lbl_title.pack()

        self.passwd_frame =tk.Label(self.passwd,text="Password",font=("Geneva",10), bg='#000000', fg='white')
        self.passwd_frame.pack()

        #Add entry box
        self.passwd_entry = Entry(self.passwd_frame, font=("Helvetica",18),show='*')
        self.passwd_entry.pack(padx=0, pady=0)

        #Add button 

        self.passwd_btn = Button(self.passwd, text="Submit",command=self.check_login)
        self.passwd_btn.pack(padx=20, pady=20)
    
    def check_login(self):
        if len(self.passwd_entry.get()) != 0 :
            self.bool = self.db.loginCheck(self.passwd_entry.get())

            if self.bool == True:
                self.passwd_entry.delete(0,END)
                self.passwd.destroy()
                messagebox.showinfo("Login","Login Successful")
                self.openwindow(HomePage)
            else:
                self.passwd_entry.delete(0,END)
                messagebox.showwarning("Login","Login Failed")
        else:
            messagebox.showwarning("Login","This Field Cannot Be Empty")

    def openwindow(self,_class):
        self.openwindow = Toplevel(self.master)
        _class(self.openwindow)
    
    def sign_up(self):
        self.sign_up = Toplevel(self.master)
        self.sign_up.title("Sign_Up")
        self.sign_up.config(background='#000000')
        self.sign_up.geometry("400x200")
         
        #Create Label Frame
        self.ll_title = tk.Label(self.sign_up,text="Password", font=("Geneva",10), bg='#000000', fg='white')
        self.ll_title.pack()
        self.sign_up_frame =tk.Label(self.sign_up,text="Password",font=("Geneva",10), bg='#000000', fg='white')
        self.sign_up_frame.pack()

        #Add entry box
        self.sign_up_entry = Entry(self.sign_up_frame, font=("Helvetica",18),show='*')
        self.sign_up_entry.pack(padx=0, pady=0)

        #Confirm Password
        
        self.sign_cf_frame =tk.Label(self.sign_up,text="Confirm Password",font=("Geneva",10), bg='#000000', fg='white')
        self.sign_cf_frame.pack() 
        self.sign_cf_frame =tk.Label(self.sign_up,text="Confirm Password",font=("Geneva",10), bg='#000000', fg='white')
        self.sign_cf_frame.pack() 

        self.sign_cf_entry = Entry(self.sign_cf_frame, font=("Helvetica",18),show='*')
        self.sign_cf_entry.pack(padx=0, pady=0)       
        #Add button 
        self.sign_cf_btn = Button(self.sign_up, text="Submit",command=self.add_master_login)
        self.sign_cf_btn.pack(padx=20, pady=20)
    
    def confirm(self):
        if self.sign_up_entry.get() == self.sign_cf_entry.get():
            return True
        return False
    
    def add_master_login(self):
        if len(self.sign_up_entry.get()) != 0 and len(self.sign_cf_entry.get()) != 0:
            if self.confirm():
                self.db.insertIntoTable(self.sign_up_entry.get())
                self.sign_up_entry.delete(0,END)
                self.sign_up.destroy()
                messagebox.showinfo("Register","Register Successful !!!")
            else:
                self.sign_up_entry.delete(0,END)
                self.sign_cf_entry.delete(0,END)
                messagebox.showwarning("Error","Passwords Are Not The Same \n\tPlease Retry")   
        else:
            self.sign_up_entry.delete(0,END)
            self.sign_cf_entry.delete(0,END)
            messagebox.showwarning("Login","These Fields Cannot Be Empty")


In [346]:
class HomePage:
#hide
    def __init__(self,master) -> None:
        self.master = master
        self.master.title("Home Page")
        self.master.geometry("970x500")
        self.master.minsize (970,500)
        self.master.maxsize (970,500)
        self.master.config(background='#dcdcdc')
        
    #Fiels for Add entry
        self.frame_fields = Frame(self.master)
        self.fieldlbl = LabelFrame(self.frame_fields,text="Record")
        self.fieldlbl.pack(fill="x", expand="yes",padx=40)
        self.frame_fields.grid(row=0,column=0)

    #SITE NAME
        self.hostname_text = StringVar()
        self.hostname_label = Label(self.fieldlbl, text='hostname', font=('bold', 12))
        self.hostname_label.grid(row=1, column=0,  padx=10, pady=10)
        self.hostname_entry = Entry(self.fieldlbl, textvariable=self.hostname_text)
        self.hostname_entry.grid(row=1, column=1,  padx=10, pady=10)

    #SITE URL
        self.url_text = StringVar()
        self.url_label = Label(self.fieldlbl, text='Url', font=('bold', 12))
        self.url_label.grid(row=1, column=2,  padx=10, pady=10)
        self.url_entry = Entry(self.fieldlbl, textvariable=self.url_text)
        self.url_entry.grid(row=1, column=3, padx=10, pady=10)
   
   #PASSWORD
        self.pwd_text = "facultatif field"
        self.pwd_label = Label(self.fieldlbl, text='Password', font=('bold', 12))
        self.pwd_label.grid(row=1, column=4,  padx=10, pady=10)
        self.pwd_entry = Entry(self.fieldlbl, textvariable=self.pwd_text)
        self.pwd_entry.grid(row=1, column=5, padx=10, pady=10)    

    #TABLE
        #Add some Style
        self.style = Style()

        #Pick a theme
        self.style.theme_use ('default')

        #Configure the Treewiew Colors
        self.style.configure("Treeview",background="#D3D3D3",foreground="black",rowheight=25,fieldbackground="#D3D3D3")

        #Create a Treeview Frame
        self.frame_tree=Frame(self.master)
        self.frame_tree.grid(row=0,padx=10,pady=5)

        #Create a Treeview Scrolbar
        self.tree_scroll = Scrollbar(self.frame_tree)
        self.tree_scroll.pack(side="right", fill="y")

        #Create The Treeview
        self.my_tree = Treeview(self.frame_tree, yscrollcommand=self.tree_scroll.set, selectmode="extended" )
        self.my_tree.pack()

        #Config the Scrollbar
        self.tree_scroll.configure(command=self.my_tree.yview)

        #Define columns
        self.my_tree['columns'] = ("ID","Site Name", "Url", "Password")

        #Format colums
        self.my_tree.column("#0", width=0, stretch=NO)
        self.my_tree.column("ID", anchor=CENTER, width=140)
        self.my_tree.column("Site Name",anchor=CENTER, width=140)
        self.my_tree.column("Url",anchor=CENTER, width=200)
        self.my_tree.column("Password",anchor=CENTER, width=200)

        #Create Headings

        self.my_tree.heading("#0", text='', anchor=CENTER)
        self.my_tree.heading("ID", text='ID', anchor=CENTER)
        self.my_tree.heading("Site Name", text='Site Name', anchor=CENTER)
        self.my_tree.heading("Url", text='Url', anchor=CENTER)
        self.my_tree.heading("Password", text='Password', anchor=CENTER)

        #Create Striped row Tags 
        self.my_tree.tag_configure ('oddrow', background="white")
        self.my_tree.tag_configure ('evenrow', background="lightblue")

        #Add Data to the screen
        self.sdb = appDb()
        self.query_DB()

    #Field for Add button
        self.frame_btns = Frame(self.master)
        self.fieldbtn = LabelFrame(self.frame_btns,text="Commands")
        self.fieldbtn.pack(fill="x", expand="yes",padx=40,pady=20)
        self.frame_fields.grid(row=1,column=0)
        self.frame_btns.grid(row=2, column=0)
        self.add_btn = Button(self.fieldbtn, text='Add Website', width=15,command=self.add_record)
        self.add_btn.grid(row=0, column=0)
        self.search_btn = Button(self.fieldbtn, text='Clear Entry Boxes',width=19,command=self.clear_entries)
        self.search_btn.grid(row=0, column=3)
        self.search_btn = Button(self.fieldbtn, text='Remove One Selected',width=19,command=self.remove_on)
        self.search_btn.grid(row=0, column=1)
        self.search_btn = Button(self.fieldbtn, text='Remove All Record',width=20,command=self.remove_all)
        self.search_btn.grid(row=0, column=2,pady=10)
    
    #Bind the Treeview
        self.my_tree.bind("<ButtonRelease-1>",self.select_record)

#Select Record
    def select_record(self,e):
        #Clear entry box
        self.hostname_entry.delete(0,END)
        self.url_entry.delete(0,END)
        self.pwd_entry.delete(0,END)

        #Grab Record Number
        self.selected = self.my_tree.focus()
        
        #Grab record values
        self.values = self.my_tree.item(self.selected,'values')
        self.hostname_entry.insert(0,self.values[1])
        self.url_entry.insert(0,self.values[2])
        self.pwd_entry.insert(0,self.values[3])
    
    #Clear entry boxes
    def clear_entries(self):
        #Clear entry box
        self.hostname_entry.delete(0,END)
        self.url_entry.delete(0,END)
        self.pwd_entry.delete(0,END)

    #Remove One Record 
    def remove_on(self):
        self.values = self.my_tree.item(self.selected,'values')
        self.x = self.my_tree.selection()[0]
        self.my_tree.delete(self.x)
        self.sdb.deleteDataTable(self.values[1])
        messagebox.showinfo("Deleted!","Your Record Has Been Removed")
    
    #Remove all record
    def remove_all(self):
        self.response= messagebox.askyesno("WOAH!!!","This Will Delete EVERYTHING From The Table\nAre You Sure?!")
        if self.response == 1 :
            for record in self.my_tree.get_children():
                self.my_tree.delete(record)
            self.sdb.dropTable()
    
    #Add new record to database
    def add_record(self):
        self.gen = generator()
        self.mdp=''
        if len(self.hostname_entry.get()) != 0 and len(self.url_entry.get()) != 0:

            if self.pwd_entry.get() == None:
                self.mdp = self.gen.passgen()
            else:
                self.mdp = self.pwd_entry.get()
            self.sdb.insertDataTable(self.hostname_entry.get(),self.url_entry.get(),self.gen.passgen())
        else:
            messagebox.showwarning("Error !!","The Site Name and Url fields must be completed")
        self.hostname_entry.delete(0,END)
        self.url_entry.delete(0,END)
        self.pwd_entry.delete(0,END)

        #Clear The Treeview Table

        self.my_tree.delete(*self.my_tree.get_children())
        self.query_DB()

    #Add Data to the screen
    def query_DB(self): 

        for record in self.my_tree.get_children():
            self.my_tree.delete(record)       
        global count , cmp
        count = 0
        cmp = 1
        #viewpass = sdb.searchPasswd()
        View = self.sdb.printData()

        for record in View:
            if count % 2 != 0:
                self.my_tree.insert(parent='', index='end', text='', values=(cmp,record[0], record[1],self.sdb.searchPasswd(record[0])),tags=('oddrow',))
            else:
                self.my_tree.insert(parent='', index='end', text='', values=(cmp,record[0], record[1],self.sdb.searchPasswd(record[0])),tags=('evenrow',))
        #Increment counter
            count += 1
            cmp += 1
    
    #Add a menu
        #Option menu
        self.my_menu = Menu(self.master) 
        self.master.config(menu=self.my_menu)
        self.option_menu = Menu(self.my_menu,tearoff=0)
        self.my_menu.add_cascade(label="Options",menu=self.option_menu)
        self.option_menu.add_command(label="Primary Color",command=self.primary_color)
        self.option_menu.add_command(label="Secondary Color",command=self.secondary_color)
        self.option_menu.add_command(label="Highlight Color",command=self.highlight_color)
        self.option_menu.add_separator()
        self.option_menu.add_command(label="Exit",command=self.close)

        #Search Menu
        self.search_menu = Menu(self.my_menu,tearoff=0)
        self.my_menu.add_cascade(label="Search",menu=self.search_menu)
        self.search_menu.add_command(label="search",command=self.lookup_records)
        self.option_menu.add_separator()
        self.search_menu.add_command(label="reset",command=self.query_DB)

    def close(self):
        self.master.destroy()
    def lookup_records (self):
        self.search = Toplevel(self.master)
        self.search.title ("Lookup Records")
        self.search.geometry("400x200")

        #Create Label Frame
        self.search_frame = Labelframe(self.search,text="Site Name")
        self.search_frame.pack(padx=10, pady=10)

        #Add entry box
        self.search_entry = Entry(self.search_frame, font=("Helvetica",18))
        self.search_entry.pack(padx=20, pady=20)

        #Add button 

        self.search_btn = Button(self.search, text="Search Records",command=self.search_record)
        self.search_btn.pack(padx=20, pady=20)
    
    def search_record(self):
        self.sum = []
        self.lookup_records = self.search_entry.get()
        #close the search box
        self.search.destroy()
        for record in self.my_tree.get_children():
            self.my_tree.delete(record)
        
        count = 0
        cmp = 1
        Vs = self.sdb.selectData(self.lookup_records)

        for record in Vs:
            if count % 2 != 0:
                self.my_tree.insert(parent='', index='end', text='', values=(cmp,record[0], record[1],self.sdb.searchPasswd(record[0])),tags=('oddrow',))
            else:
                self.my_tree.insert(parent='', index='end', text='', values=(cmp,record[0], record[1],self.sdb.searchPasswd(record[0])),tags=('evenrow',))
        #Increment counter
            count += 1
            cmp += 1

    def primary_color(self):
        self.primary_color = colorchooser.askcolor()[1]

        if self.primary_color:
               #Create Striped row Tags 
            self.my_tree.tag_configure ('evenrow', background=self.primary_color)
       
    def secondary_color(self):
        self.secondary_color = colorchooser.askcolor()[1]

        if self.secondary_color:
            self.my_tree.tag_configure ('oddrow', background=self.secondary_color)

    def highlight_color(self):
        self.highlight_color = colorchooser.askcolor()[1]
        if self.highlight_color:
            #Change de select color
            self.style.map('Treeview',background=[('selected',self.highlight_color)])



In [214]:
root = Tk()
app = StartPage(root)
root.mainloop()